<br>

# createWindEpiDatasets #
<br>

**Author:** Andrew Larkin <br>
**Summary:** Join selected matches, wind metrics, and vital statistics into a single dataset and reformat for linear and logistic regression analyses

### load required libraries and define global constants ###

In [4]:
import pandas as ps
import os
# python file containing absolute filepaths to directories containing sensitivite information
import gConst as const 
print(const.sampleNumber)
print(const.matchQuality)

[1, 4]
[15, 25, 50, 100]


## helper functions used in main script ##

### load vital statistics and wind datasets into memory ###
**INPUTS:**
 - vitalStatsFilepath (str) - absolute filepath to vital stats data
 - exposedFilepath (str) - absolute filepath to wind metrics for exposed
 - controlFilepath (str) - absolute filepath to wind metrics for controls
 - allWindFilepath (str) - absolute filepath to wind metrics for all residences within 500m of high traffic roads
 - unrestrictedMatchFilepath (str array) - absolute filepaths to exposed/control matches for 15,25,50, and 100m
 - restrictedMatchFilepath (str array) - absolute filepaths to exposed/control matches for 15,25, 50, and 100m, restricted to 37 to 42 weeks
<br>

**OUTPUTS:**<br>
 - arraay consisting of 5 pandas dataframes, one dataframe for each input filepath.  Dataframe are in the same order as the inputs

In [3]:
def loadData(vitalStatsFilepath,exposedFilepath,controlFilepath,allWindFilepath,
             unrestrictedMatchFilepath,restrictedMatchFilepath):
    
    # load datasets
    vitalData = ps.read_csv(vitalStatsFilepath)
    exposedData = ps.read_csv(exposedFilepath)
    controlData = ps.read_csv(controlFilepath)
    allWindData = ps.read_csv(allWindFilepath)
    unrestrictedMatchData = []
    
    # there's one match file for 15,25,50, and 100m distances
    for matchFile in unrestrictedMatchFilepath:
        unrestrictedMatchData.append(ps.read_csv(matchFile))
    restrictedMatchData = []
    
    # there's one match file for 15,25,50, and 100m distances
    for matchFile in restrictedMatchFilepath:
        restrictedMatchData.append(ps.read_csv(matchFile))
    
    vitalData = ps.read_csv(vitalStatsFilepath)
    
    print(
        """n records for \nvitalStats: %i\nexposedData: %i\ncontrolData %i\nallWindData: 
        %i\nunrestrictedMatchData: %i\nrestrictedMatchData: %i"""
        %(vitalData.count()[0],exposedData.count()[0],controlData.count()[0],allWindData.count()[0],
          unrestrictedMatchData[0].count()[0],restrictedMatchData[0].count()[0])
    )
    return([vitalData,exposedData,controlData,allWindData,unrestrictedMatchData,restrictedMatchData])

### inner join match and wind metrics for the exposed quartile.  Add number of matches for each id and exposure group indicator variable ###
**INPUTS:**
 - matchData (pandas DataFrame) - records for the exposed/control wind matching
 - exposedData (pandas DataFrame) - wind metrics for the exposed group (top quartile of wind exposures)
<br>

**OUTPUTS:**<br>
 - pandas dataframe containing joined wind and matched metrics for the exposed group

In [4]:
def mergeExposedData(matchData,exposedData):
    
    # rename variables so they will be the same for exposed and control records
    matchData['wind_dist'] = matchData['exp_dist']
    matchData['uniqueid'] = matchData['exp_id']
    matchData['matchid'] = matchData['ctrl_id']
    matchData['near_dist'] = matchData['near_dist_exp']
    matchData = matchData[['wind_dist','uniqueid','matchid','near_dist']]
  
    # remove exposed records that weren't matched
    matchedExposedIds = list(set(matchData['uniqueid']))
    matchedExposedInfo = exposedData[exposedData['uniqueid'].isin(matchedExposedIds)]
 
    # inner join the exposed wind metrics and match data. 
    exposedMatchJoin = matchedExposedInfo.merge(matchData,how='inner',on='uniqueid')
    exposedMatchJoin['windCat'] = 0
    print(
        "number of input exposed records: %i, number of output exposed records: %i\n" 
          %(exposedData.count()[0],exposedMatchJoin.count()[0])
    )
    return(exposedMatchJoin)

### inner join match and wind metrics for the control gruop.  Add number of matches for each id and control group indicator variable ###
**INPUTS:**
 - matchData (pandas DataFrame) - records for the exposed/control wind matching
 - controlData (pandas DataFrame) - wind metrics for the control group (bottom quartile of wind exposures)
<br>

**OUTPUTS:**<br>
 - pandas dataframe containing joined wind and matched metrics for the control group

In [10]:
def mergeControlData(matchData,controlData):
    
    # rename variables so they will be the same for exposed and control records
    matchData['wind_dist'] = matchData['ctrl_dist']
    matchData['uniqueid'] = matchData['ctrl_id']
    matchData['matchid'] = matchData['ctrl_id']
    matchData['near_dist'] = matchData['near_dist_ctrl']
    matchData = matchData[['wind_dist','uniqueid','matchid','near_dist']]
  
    
    
    # remove control records that weren't matched
    matchedControlIds = list(set(matchData['uniqueid']))
    matchedControlInfo = controlData[controlData['uniqueid'].isin(matchedControlIds)]
    
    # inner join the control wind metrics and match data. 
    controlMatchJoin = matchedControlInfo.merge(matchData,how='inner',on='uniqueid')
    controlMatchJoin['windCat'] = 1
    print(
        "number of input control records: %i, number of output control records: %i\n" 
          %(controlData.count()[0],controlMatchJoin.count()[0])
    )
    return(controlMatchJoin)

### inner join vital stats with wind metrics for all residences within 500m of high traffic roads ###
**INPUTS:**
 - vitalStatsData (pandas DataFrame) - all vital stats variables
 - windData (str) - all wind metrics, updated wit exposure categories and n matches
<br>

**OUTPUTS:**<br>
 - pandas DataFrame containing vitalStats joined to wind records

In [1]:
def joinVitalAndWind(vitalData,windData):
    
    # drop columns already present in the vital stats databasea
    windData = windData.drop(
        ['b_es_ges', 'b_long', 'b_lat','byear', 'bmonth', 'bday', 'bdate'],axis=1
    )
    
    windVitalJoined = vitalData.merge(windData,how='inner',on='uniqueid')  
    
    return(windVitalJoined)

### combine exposed, control, and vital statistics records together ###
**INPUTS:**
 - exposedCohort (pandas dataframe) - contains exposed matches with wind metrics
 - controlCohort (pandas dataframe) - contains control matches with wind metrics
 - vitalData (pandas dataframe) - vital statistics records for the cohort
<br>

**OUTPUTS:**<br>
 - pandas dataframe containing match, wind, and vital statistics data

In [3]:
def combineExposedControl(exposedCohort,controlCohort,vitalData):
    combinedCategories = []
    
    # for each match criteria threshold (15,25,50,100m), join match, wind, and vital statistics datasets
    for index in range(len(exposedCohort)):
        tempData = ps.concat([
            exposedCohort[index],
            controlCohort[index],
        ])
        
        windWithVitals = joinVitalAndWind(vitalData,tempData)
        
        combinedCategories.append(windWithVitals)
    return(combinedCategories)

<br>

## main script  ##
<br>
<br>

### load datasets into memory ###

In [20]:
vitalData,exposedData,controlData,allWindData,unrestrictedMatchData,restrictedMatchData = loadData(
    const.VITAL_STATS_FILEPATH, const.EXPOSED_FILEPATH,const.CONTROL_FILEPATH,const.WIND_METRICS,
    const.SELECTED_MATCH_FILES,const.SELECTED_MATCH_FILES_RESTRICT
)

### join about exposed match data with wind metrics ###

In [ ]:
exposedWithMatchesUnrestricted,exposedWithMatchesRestricted = [],[]
for index in range(len(unrestrictedMatchData)):
    exposedWithMatchesUnrestricted.append(mergeExposedData(unrestrictedMatchData[index],exposedData))
    exposedWithMatchesRestricted.append(mergeExposedData(restrictedMatchData[index],exposedData))

### join about exposed match data with wind metrics ###

In [ ]:
controlWithMatchesUnrestricted,controlWithMatchesRestricted = [],[]
for index in range(len(unrestrictedMatchData)):
    controlWithMatchesUnrestricted.append(mergeControlData(unrestrictedMatchData[index],controlData))
    controlWithMatchesRestricted.append(mergeControlData(restrictedMatchData[index],controlData))

### join exposed and control datasets, now including wind metrics ###

In [ ]:
combinedCohortUnrestricted = combineExposedControl(exposedWithMatchesUnrestricted,controlWithMatchesUnrestricted,vitalData)
combinedCohort37to42 = combineExposedControl(exposedWithMatchesRestricted,controlWithMatchesRestricted,vitalData)

### save records to csv ###

In [ ]:
# keep track of location in array.  Arrays hold epi data for all sample/match criteria combinations
index = 0

# for 1:1 and 1:4 matching:
for sampleCrit in const.sampleNumber:
    
    # for each match criteria in (15,25,50,100), 
    for matchCrit in const.matchQuality:
        
        # save unrestricted records
        filename = const.EPI_FOLDER + "wind_epi_" + str(sampleCrit) + "_" + str(matchCrit) + "_" + "all_Jan27.csv"
        if not(os.path.exists(filename)):
            combinedCohortUnrestricted[index].to_csv(filename,index=False)
            
        # save records restricted from 37 to 42 weeks
        filename = const.EPI_FOLDER + "wind_epi_" + str(sampleCrit) + "_" + str(matchCrit) + "_" + "37to42_Jan27.csv"
        if not(os.path.exists(filename)):
            combinedCohort37to42[index].to_csv(filename,index=False)
        index+=1
        print("finished sampleCrit %i and matchCrit %i" %(sampleCrit,matchCrit))